In [311]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import functools
from dataclasses import dataclass
import time
from tqdm.auto import tqdm

logging.basicConfig(level=logging.DEBUG)

In [287]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

@dataclass
class path:
    genome: np.ndarray
    fitness: float = None

def generate_cycle(cities):
  genome = [0]
  tour = np.arange(1,len(cities))
  np.random.shuffle(tour)
  genome.extend(tour)
  genome.extend([0])
  genome=np.array(genome)

  individual = path(genome)
  individual.fitness = -tsp_cost(individual.genome)

  return individual

def parent_selection(population):
  index = np.random.randint(len(population))
  return population[index]

In [306]:
def swap(individual, indexes):
  a = individual.genome[indexes[0]]
  individual.genome[indexes[0]] = individual.genome[indexes[1]]
  individual.genome[indexes[1]] = a
  return individual

def mutation(sol, rate=0.1):
    new_solution = path([])
    new_solution.genome = sol.genome.copy()
    mutated = False

    while not mutated or np.random.random() < rate:
      indexes = np.zeros(2)
      while indexes[0]==indexes[1]:
        indexes = np.random.choice(len(new_solution.genome)-2, size=2)
      indexes = indexes+1
      new_solution = swap(new_solution, indexes)
      mutated = True

    new_solution.fitness = -tsp_cost(new_solution.genome)
    return new_solution

def cycle_xover(p_1, p_2):
  indexes = np.zeros(2)
  while indexes[0]==indexes[1]:
    indexes = np.random.choice(len(p_1.genome)-2, size=2)
  indexes = indexes+1

  mask = np.zeros(len(p_1.genome), dtype=np.bool_)
  for i in range(len(mask)):
    if i in range(indexes.min(),indexes.max()+1):
      mask[i] = True

  genome = np.zeros(len(p_1.genome), dtype=np.int8)
  genome[mask] = p_1.genome[mask]

  remaining_genes = []
  for i in range(len(p_2.genome)):
    if p_2.genome[i] not in(p_1.genome[mask]):
      remaining_genes.append(p_2.genome[i])

  selected = 0
  for i in range(len(genome)):
    if genome[i]==0:
      genome[i]=remaining_genes[selected]
      selected += 1

  child = path(genome)
  child.fitness = -tsp_cost(child.genome)

  return child

## Evolutionary Algorithm


##Italy

In [308]:
CITIES = pd.read_csv('/content/Cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

POPULATION_SIZE = 10
population = [generate_cycle(CITIES) for _ in range(POPULATION_SIZE)]

In [310]:
max_generations = 1000
offspring_size = 5

for g in tqdm(range(max_generations)):
    offspring = list()
    for _ in range(offspring_size):
        # HYPERMODERN
        if np.random.random() < .5:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = cycle_xover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

print(f"Final Path: lenght = {-population[0].fitness:.2f}km")

  0%|          | 0/1000 [00:00<?, ?it/s]

Final Path: lenght = 6357.14km


## Bunus: Hill Climb

In [ ]:
# Function used to perform Simulated Annealing
def S_A(a,b,temp=1):
    Max = max(a,b)
    Min = min(a,b)
    return np.exp(-(Max-Min)/temp)

def tsp_climb(steps=250, num_samples=5, prob=0.1, temperature=2, plot=False):
    #generate a random initial solution
    initial_solution = generate_cycle(CITIES)

    history = []
    current_solution = path([])
    current_solution.genome = initial_solution.genome.copy()
    current_solution.fitness = -tsp_cost(current_solution.genome)
    best_solution = current_solution

    # Hill Climb
    start = time.time()
    for i in tqdm(range(steps)):
        best_so_far = path([])
        best_so_far.genome = current_solution.genome.copy()
        best_so_far.fitness = -tsp_cost(best_so_far.genome)

        # generate candidate steps
        for _ in range(num_samples):
            temp_sol = mutation(current_solution, rate=prob)
            if temp_sol.fitness > best_so_far.fitness:
                best_so_far = temp_sol
            elif temperature!=None and np.random.random() < S_A(temp_sol.fitness, best_so_far.fitness, temp=temperature):
                best_so_far = temp_sol

        current_solution = best_so_far
        if current_solution.fitness > best_solution.fitness:
            best_solution = current_solution

    end = time.time()

    # Plot path graph
    # if plot==True:


    #Plot some results
    print(f"Initial Path: lenght = {-initial_solution.fitness:.2f}km")
    print(f"Final Path: lenght = {-best_solution.fitness:.2f}km")
    print(f"Executed in {end-start:.4} seconds")
    return best_solution

##Italy (Hill Climb)

In [205]:
CITIES = pd.read_csv('/content/Cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# CITIES.head()

italy_sol = tsp_climb(steps=500, num_samples=5, prob=0.1)

  0%|          | 0/500 [00:00<?, ?it/s]

Initial Path: lenght = 21516.56km
Final Path: lenght = 6697.15km
Executed in 0.3043 seconds


##Cina (Hill Climb)

In [ ]:
CITIES = pd.read_csv('/content/Cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# CITIES.head()

china_sol = tsp_climb(steps=500, num_samples=5, prob=0.1)

  0%|          | 0/500 [00:00<?, ?it/s]

Initial Path: lenght = 996953.96km
Final Path: lenght = 612023.31km
Executed in 2.074 seconds


In [ ]:
china_sol = tsp_climb(steps=100_000, num_samples=5, prob=0.1)

  0%|          | 0/100000 [00:00<?, ?it/s]

Initial Path: lenght = 990190.41km
Final Path: lenght = 205111.61km
Executed in 329.1 seconds
